In [69]:
from PIL import Image, ImageFilter
import numpy as np
import glob
from pathlib import Path


In [77]:
class LastAvg:
    def __init__(self):
        self.last = []
    def clean(self):
        self.last = []
    def add(self, x):
        self.last.append(x)
        if len(self.last) >= 10:
            self.last = self.last[1:]
    def average(self):
        return sum(self.last) / len(self.last)

def preprocess_image(path):
    image = Image.open(path)
    image.show()
    width = image.width
    height = image.height
    # correct full masks
    for j in range(width):
        for i in range(height):
            r, g, b = image.getpixel((j, i)) 
            v = min([r,g,b])
            bm = (r == g and g != b) # blue mask
            rm = (g == b and g != r) # red mask
            gm = (r == b and r != g) # green mask
            if bm or rm or gm:
                v = r
                if rm: v = g
                image.putpixel((j,i), (v, v, v))
    # for j in range(width):
    #     for i in range(height):
    #         r, g, b = image.getpixel((j, i))
    #         bm = (b > g and b > r) # blue mask
    #         rm = (r > g and r > b) # red mask
    #         gm = (g > b and g > r) # green mask
    #         if bm or rm or gm:
    #             v = min([r,g,b])
    #             image.putpixel((j,i), (v, v, v))
    
    ni = np.array(image)
    r = ni[:,:,0]
    g = ni[:,:,1]
    b = ni[:,:,2]
    mask = ((r != g) | (g != b)) * 255
    mask = Image.fromarray(mask.astype(np.uint8))
                
    for j in range(width):
        for i in range(height):
            r, g, b = image.getpixel((j, i)) 
            bm = b >= g and b >= r # blue mask
            rm = r >= g and r >= b # red mask
            gm = g >= r and g >= b # green mask
            if bm or rm or gm:
                if rm: v = (g + b) / 2
                if gm: v = (r + b) / 2
                if bm: v = (r + g) / 2
                v = round(v)
                image.putpixel((j,i), (v, v, v))
                
    # image.show()
    
    filter = ImageFilter.GaussianBlur(radius=2)
    gray = image.convert('L')
    gray_blur = gray.filter(filter)
    prevBlock = None
    prevGray = None
    average_outer = LastAvg()
    average_inner = LastAvg()

    for j in range(width):
        for i in range(height):
            block = mask.getpixel((j, i)) == 255
            current_gray = gray.getpixel((j, i))
            if not block and prevBlock is not None: # leave mask
                midGrayAround = (current_gray + average_outer.average()) / 2
                midGrayIn = average_inner.average()
                delta = midGrayAround - midGrayIn
                for sub_i in range(prevBlock, i):
                    c = gray.getpixel((j, sub_i))
                    new_gray = round(c + delta)
                    gray.putpixel((j, sub_i), new_gray)
                prevBlock = None
                prevGray = None
            elif block and prevBlock is None: # enter mask
                prevBlock = i
            
            if block:
                average_inner.add(current_gray)
            
            if not block:
                average_outer.add(current_gray)

    ###########
    prevBlock = None
    prevGray = None
    average_outer = LastAvg()
    average_inner = LastAvg()

    for i in range(height):
        for j in range(width):
            block = mask.getpixel((j, i)) == 255
            current_gray = gray.getpixel((j, i))
            if not block and prevBlock is not None: # leave mask
                midGrayAround = (current_gray + average_outer.average()) / 2
                midGrayIn = average_inner.average()
                delta = midGrayAround - midGrayIn
                for sub_i in range(prevBlock, i):
                    c = gray.getpixel((j, sub_i))
                    new_gray = round(c + delta)
                    gray.putpixel((j, sub_i), new_gray)
                prevBlock = None
                prevGray = None
            elif block and prevBlock is None: # enter mask
                prevBlock = i
            
            if block:
                average_inner.add(current_gray)
            
            if not block:
                average_outer.add(current_gray)
    ###########

    return gray



In [78]:
# preprocess_image("A/A/A3/to/c1r1e0n2-0.png")
# preprocess_image("A/A/A3/to/c1r1e0n46-9.png")
# preprocess_image("A/A/A3/to/c1r1e0n45-35.png")
# preprocess_image("A/A/A3/to/c1r1e0n46-38.png")
gray = preprocess_image("A/A/A3/to/c1r1e0n45-39.png")
gray.show()


kf.service.services: KApplicationTrader: mimeType "x-scheme-handler/file" not found
kf.service.services: KApplicationTrader: mimeType "x-scheme-handler/file" not found
org.kde.kdegraphics.gwenview.lib: Unresolved mime type  "image/x-mng"
org.kde.kdegraphics.gwenview.lib: Unresolved raw mime type  "image/x-nikon-nrw"
org.kde.kdegraphics.gwenview.lib: Unresolved raw mime type  "image/x-samsung-srw"


kf.service.services: KApplicationTrader: mimeType "x-scheme-handler/file" not found
kf.service.services: KApplicationTrader: mimeType "x-scheme-handler/file" not found
org.kde.kdegraphics.gwenview.lib: Unresolved mime type  "image/x-mng"
org.kde.kdegraphics.gwenview.lib: Unresolved raw mime type  "image/x-nikon-nrw"
org.kde.kdegraphics.gwenview.lib: Unresolved raw mime type  "image/x-samsung-srw"


In [75]:
A3_path = 'A/A/A3'
A3_save_path = Path(f'{A3_path}/save')
for path in glob.glob(f'{A3_path}/to/*.png'):
    print('On', path)
    gray = preprocess_image(path)
    A3_save_path.mkdir(exist_ok=True, parents=True)
    gray.save(str(A3_save_path / Path(path).name))

On A/A/A3/to/c1r1e0n39-24.png
On A/A/A3/to/c1r1e0n17-35.png
On A/A/A3/to/c1r1e0n27-36.png
On A/A/A3/to/c1r1e0n40-20.png
On A/A/A3/to/c1r1e0n30-14.png
On A/A/A3/to/c1r1e0n11-23.png
On A/A/A3/to/c1r1e0n45-30.png
On A/A/A3/to/c1r1e0n34-23.png
On A/A/A3/to/c1r1e0n14-22.png
On A/A/A3/to/c1r1e0n24-23.png
On A/A/A3/to/c1r1e0n22-23.png
On A/A/A3/to/c1r1e0n24-1.png
On A/A/A3/to/c1r1e0n11-9.png
On A/A/A3/to/c1r1e0n26-36.png
On A/A/A3/to/c1r1e0n6-10.png
On A/A/A3/to/c1r1e0n15-9.png
On A/A/A3/to/c1r1e0n10-23.png
On A/A/A3/to/c1r1e0n15-2.png
On A/A/A3/to/c1r1e0n20-6.png
On A/A/A3/to/c1r1e0n5-38.png
On A/A/A3/to/c1r1e0n43-25.png
On A/A/A3/to/c1r1e0n31-39.png
On A/A/A3/to/c1r1e0n44-27.png
On A/A/A3/to/c1r1e0n31-4.png
On A/A/A3/to/c1r1e0n37-2.png
On A/A/A3/to/c1r1e0n46-37.png
On A/A/A3/to/c1r1e0n27-11.png
On A/A/A3/to/c1r1e0n19-2.png
On A/A/A3/to/c1r1e0n18-16.png
On A/A/A3/to/c1r1e0n27-10.png
On A/A/A3/to/c1r1e0n3-19.png
On A/A/A3/to/c1r1e0n27-21.png
On A/A/A3/to/c1r1e0n28-31.png
On A/A/A3/to/c1r1e0n1